Name: Suman Roy

Roll Number: MDS202041

NLP Assignment 03

In [3]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import word2vec
import logging
from tqdm import tqdm
from scipy import stats
import numpy as np
from fastdist import fastdist

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocessing

We have already extracted the texts from JSON files and discarded the non-english documents in Assignment 1. Here we will use the primary text corpus file that was created just after extracting the texts from the JSON files and discarding non-english documents(Before preprocessing). We will work with only the English documents. However, in Assignment 1 we did not consider the sentence structures. Here, we will preprocess the text corpus keeping the sentence structure intact. 

In [41]:
with open('/content/drive/MyDrive/CMI/Semester 4/NLP/Assignment 1/corpus_english.txt','r') as f:
    text=f.read()
text_list=text.split("\n")
N=len(text_list)
print("Number of documents: ",N)

Number of documents:  54839


In [38]:
def decontracted(text):
    # specific
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)

    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def delete_single_alphabet(text):
  # function to remove single character except 'a' like 'b','x' etc.)
  pattern = re.compile(r' [b-zB-Z] ')
  return re.sub(pattern, ' ', text)

def delete_decimal(text):
  # function to remove decimal numbers because they can cause nuisance in sentence tokenization
  pattern = re.compile(r'[\d*\.?\d+]')
  return re.sub(pattern, ' ', text)

def delete_digit(text):
  # function to remove digits
  pattern = re.compile(r'\d')
  return re.sub(pattern, '', text)

def delete_foreign_characters(text):
  #function to remove foreign non-ascii characters
  pattern= re.compile(r'[^\x00-\x7F]+')
  return re.sub(pattern,' ',text)
  
def delete_parenthesis(text):
  #function to remove parenthesis and text inside parenthesis
  pattern = re.compile(r'[\[\(\{]+[^\n\[\(\{]*[\)\]\}]+')
  return re.sub(pattern, ' ', text)

def delete_html_address(text):
  #function to remove the html addresses
  pattern= re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&\/\/=]*)')
  return re.sub(pattern,' ',text)

def delete_email(text):
  #function to remove email addresses
  pattern= re.compile(r' [a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+ ')
  return re.sub(pattern,' ',text)

def delete_punctuations(text):
  #function to remove punctuations which are not the delimiters of the sentences (.!?)
  pattern= re.compile(r'''[\[\]\(\)\{\}`^'",:;~|_#$%&><=*\\\/\+\-\@]''')
  return re.sub(pattern,' ',text)

def delete_white_space(text):
  #function to remove extra white spaces
  pattern= re.compile(r'(?<= ) +')
  return re.sub(pattern,'',text)

def lemmatize(text):
  #function to lemmatize the words
  lemmatizer = WordNetLemmatizer()
  x = word_tokenize(text)
  y = [lemmatizer.lemmatize(word) for word in x]
  text_=" ".join(y)
  return text_



In [42]:
def preprocess(text):
  text=decontracted(text)
  text=delete_single_alphabet(text)
  text=delete_decimal(text)
  text=delete_digit(text)
  text=delete_foreign_characters(text)
  text=delete_parenthesis(text)
  text=delete_html_address(text)
  text=delete_email(text)
  text=delete_punctuations(text)
  text=delete_white_space(text)
  text=lemmatize(text)
  return text

We are randomly selecting 50 documents from which we would get the sentences to be used for sentence embedding. We are preprocessing only those documents. Rest of the documents have been already preprocessed without keeping sentence structures intact in assignment 1. 

In [51]:
import random
random_doc_index=random.sample(range(0, N), 50)
print(random_doc_index)

[50975, 27424, 41694, 10426, 33613, 24352, 39039, 33798, 43348, 20531, 45878, 50316, 7540, 30893, 50576, 21568, 13905, 22936, 36997, 42704, 2593, 39364, 9800, 24138, 24683, 5165, 13940, 4970, 36990, 40031, 183, 38031, 45289, 51460, 45404, 12109, 53975, 27478, 30624, 1981, 22370, 37524, 39518, 8118, 3665, 44428, 5343, 38456, 9418, 22074]


In [52]:
corpus=''
for i in tqdm(random_doc_index):
  if len(corpus)!=0:
    corpus=corpus+'\n'+preprocess(text_list[i])
  else:
    corpus=corpus+preprocess(text_list[i])
with open('/content/drive/MyDrive/CMI/Semester 4/NLP/Assignment 3/corpus_english_sentence.txt','a') as f:
  f.write(corpus)

100%|██████████| 50/50 [00:00<00:00, 74.66it/s]


### Getting the sentences from 50 random documents

In [53]:
with open('/content/drive/MyDrive/CMI/Semester 4/NLP/Assignment 3/corpus_english_sentence.txt','r') as f:
    text=f.read()
text_list=text.split("\n")
N=len(text_list)
print("Number of documents: ",N)

Number of documents:  50


In [54]:
sentences={}
i=0
for text in text_list:
  try:
    for sentence in sent_tokenize(text):
      if len(sentence.split())>5 and len(sentence.split())<20: #only accepting the sentences between length 5 and 20
        sentences[i]=sentence
        i=i+1
  except:
    pass

In [55]:
len(sentences) #total number of sentences to work with

5204

### Creating the corpus for calculating word vectors

In [13]:
def preprocess_2(text):
  text=lemmatize(text)
  return text

We had already built a corpus in assignment 1 after cleaning by regex(before stopwords removal). We would use that courpus and lemmatize the documents and use it for the word vector calculation in this assignment. 

In [12]:
with open('/content/drive/MyDrive/CMI/Semester 4/NLP/Assignment 1/corpus_english_preprocess.txt','r') as f:
    text=f.read()
text_list=text.split("\n")
N=len(text_list)
print("Number of documents: ",N)

Number of documents:  54839


In [14]:
for i in [0,5000,10000,15000,20000,25000,30000,35000,40000,45000,50000]:
  corpus=""
  j=min(i+5000,N)
  for text in tqdm(text_list[i:j]):
    if len(corpus)!=0:
      corpus=corpus+'\n'+preprocess_2(text)
    else:
      corpus=corpus+preprocess_2(text)
  if i!=50000:
    corpus=corpus+'\n'
  with open('/content/drive/MyDrive/CMI/Semester 4/NLP/Assignment 3/corpus_english_preprocess_final.txt','a') as f:
    f.write(corpus)

100%|██████████| 4839/4839 [05:55<00:00, 13.61it/s]


### Compute word vectors

Here we are computing word vectors of size 50 and considering 10 epochs. 

In [15]:
def compute_word_vectors(corpus_file_path,model_path,size=50): 
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    text = word2vec.Text8Corpus(corpus_file_path)
    model = word2vec.Word2Vec(text, size=size, window=5, min_count=1, workers=10, iter=10)
    model.save(model_path)
    return model

In [16]:
corpus_file_path='/content/drive/MyDrive/CMI/Semester 4/NLP/Assignment 3/corpus_english_preprocess_final.txt'
model_path='/content/drive/MyDrive/CMI/Semester 4/NLP/Assignment 3/w2v.bin'

In [17]:
model = compute_word_vectors(corpus_file_path,model_path)

2022-05-19 06:47:44,697 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2022-05-19 06:47:44,751 : INFO : collecting all words and their counts
2022-05-19 06:47:44,768 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-19 06:48:42,086 : INFO : PROGRESS: at sentence #10000, processed 100000000 words, keeping 416795 word types
2022-05-19 06:49:31,625 : INFO : PROGRESS: at sentence #20000, processed 200000000 words, keeping 639477 word types
2022-05-19 06:49:31,923 : INFO : collected 640392 word types from a corpus of 200617374 raw words and 20062 sentences
2022-05-19 06:49:31,925 : INFO : Loading a fresh vocabulary
2022-05-19 06:49:35,330 : INFO : effective_min_count=1 retains 640392 unique words (100% of original 640392, drops 0)
2022-05-19 06:49:35,332 : INFO : effective_min_count=1 leaves 200617374 word corpus (100% of original 200617374, drops 0)
2022-05-19 06:49:37,901 : INFO : deleting the raw counts dictionary of 64

Loading the model:

In [ ]:
model = word2vec.Word2Vec.load("/content/drive/MyDrive/CMI/Semester 4/NLP/Assignment 3/w2v.bin")

### Sentence Vectors and RNN

In [18]:
def sentence_embedding(model,sentences,size=150,word_size=50):
  #size is sentence vector size and word_size is word vector size
  #matrix to be multiplied with the past hidden state to get new hidden state
  U=np.random.normal(0,0.01,(size,size))  
  #Weight matrix of size (word_vector_size*sentence_vector_size) to be multiplied with the wordVector to get the new hidden state
  W=np.random.normal(0,0.01,(size,word_size))
  #initial value of hidden state
  h0=np.random.normal(0,0.01,size)  
  past_h_forward=U.dot(h0)
  past_h_reverse=U.dot(h0)
  h_forward=np.zeros(size)
  h_reverse=np.zeros(size)
  vectors={}  #to store the sentence vectors

  for i in tqdm(sentences.keys()):
    #forward direction
    for word in sentences[i].split():
      if word in model.wv.vocab:
        x=model.wv.get_vector(word)
        y=stats.zscore(x)
        # h[t]=tanh(W*y+U*h[t-1])
        h_forward=np.tanh(np.dot(W,y)+np.dot(U,past_h_forward))
        past_h_forward=h_forward/size
    #reverse_direction
    for word in sentences[i].split()[::-1]:
      if word in model.wv.vocab:
        x=model.wv.get_vector(word)
        y=stats.zscore(x)
        # h[t]=tanh(W*y+U*h[t-1])
        h_reverse=np.tanh(np.dot(W,y)+np.dot(U,past_h_reverse))
        past_h_reverse=h_reverse/size
    vectors[i]=np.concatenate((h_forward,h_reverse)) #sentence vectors of each sentence
  return vectors


In [56]:
vectors=sentence_embedding(model,sentences,size=150,word_size=50)

100%|██████████| 5204/5204 [00:19<00:00, 271.05it/s]


### Result

Here we would try to show examples of similarities. A sample sentence is given as input and this function would produce 10 most similar sentences with respect to the given sentence. 

We have used **cosine distance as similarity measure** for this case. Cosine similarity measures the similarity between two vectors of an inner product space. It is measured by the cosine of the angle between two vectors and determines whether two vectors are pointing in roughly the same direction. It is often used to measure document similarity in text analysis.

The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance because of the size (like, the word ‘covid’ appeared 50 times in one document and 10 times in another) they could still have a smaller angle between them. Smaller the angle, higher the similarity.

In [57]:
len(vectors)  #total number of vectors would be same as total number of sentences

5204

In [68]:
vectors[0].shape #size of each sentence vector (2*150)

(300,)

In [73]:
def similar_sentence(sentences,vectors,index):
  sentence_list=[]
  s=np.zeros(len(sentences)) # an array to store the cosine distances of all the sentences wrt the given sentence
  i=0
  for l in range(len(sentences)):
    s[i]=fastdist.cosine(vectors[index],vectors[l]) 
    i=i+1
  ind = np.argpartition(s, -10)[-10:] #taking the highest 10 similarity scores
  print("The sentence is :", sentences[index])
  print("The indices of the sentences most similar to sentence ",index," : ",ind)
  for i in ind:
    sentence_list.append((sentences[i],s[i]))
  return (sentence_list)
  

**Showing 5 random sentences and their most similar sentences:**

In [78]:
random_sentence_index=random.sample(range(0, len(sentences)), 1)
for i in random_sentence_index:
  x=similar_sentence(sentences,vectors,i)
x

The sentence is : we are also vulnerable to infections which can easily be transmitted to children.
The indices of the sentences most similar to sentence  4404  :  [2164 3182 4829 1086 2105 1212 4934 1224 2889 4404]


[('we performed dna panel analysis for defects in beta oxidation carnitine cycle disorders and glycogenolysis.',
  0.7417873775360342),
 ('we calculated the bivariate correlation between the ht levels in csf platelets and saliva.',
  0.7417970636097072),
 ('we found that algorithm generated articles are very similar in format and content.',
  0.7418724050950952),
 ('we now present final analysis of the data.', 0.7446316171467906),
 ('we identified genomic changes mutations and polymorphisms .',
  0.7418514670065145),
 ('we selected metabolic centers where a dedicated psychologist trained in following pku patients could administer the questionnaire.',
  0.744668510969618),
 ('we used ibm spss statistics for windows version for analysis.during months critically ill patients were included in the study.',
  0.7446208406844732),
 ('methods we investigated patients with earlytreated pku aged to years .',
  0.9999992558070613),
 ('we showed that the extent of lamp lca co localization was redu

In [75]:
random_sentence_index=random.sample(range(0, len(sentences)), 1)
for i in random_sentence_index:
  x=similar_sentence(sentences,vectors,i)
x

The sentence is : ninetynine percent had health insurance with having public insurance alone.
The indices of the sentences most similar to sentence  3461  :  [2355 3962 3310 1165 1567 4563 2047 4786 3461 2443]


[('as mitochondrial disorders are rare multicentre collaboration is essential to generate large patient cohorts for high power analyses.',
  0.3734860225348559),
 ('calves fed maternal colostrum used feed more efficiently than calves fed colostrum replacement.',
  0.3774498844327065),
 ('iem as well as more internalizing and externalizing symptoms in their children.',
  0.38247932260428946),
 ('untreated or poorly treated pku can lead to intellectual disability seizures behavioural problems and mental disorders.',
  0.40195701195030425),
 ('dtx is a promising new therapeutic approach for otc deficiency.',
  0.41182297102793347),
 ('as previously stated some governments have relied on simultaneous interpretation or closed captioning of briefings relayed on electronic platforms.',
  0.4122446893558793),
 ('patient is a year old male who presented at years with indeterminate non targeted direct to consumer testing.',
  0.41731691834216034),
 ('diakopoulos identified the algorithmic power 

In [77]:
random_sentence_index=random.sample(range(0, len(sentences)), 1)
for i in random_sentence_index:
  x=similar_sentence(sentences,vectors,i)
x

The sentence is : intra day and inter day precision was acceptable .
The indices of the sentences most similar to sentence  2795  :  [1478   91  274 1981 2544 2795 3049  903 4866  904]


[('in addition exhaled no concentrations were evaluated.results data were shown as average s.d.',
  0.37310322821553393),
 ('citrate buffer for min at room temperature.', 0.37666573869532943),
 ('dci from mct prior to entering the study and per protocol received and overall tolerated mean .',
  0.37832414020237726),
 ('glucose hdl c ldl c and leptin showed no significant difference.',
  0.42151946974882276),
 ('cerebral magnetic resonance imaging showed cortical atrophy cerebellar hypoplasia and increased signal in the basal ganglia on t images.',
  0.3991395596086521),
 ('intra day and inter day precision was acceptable .', 1.0),
 ('significant attenuation of the rate of decline in ml score mean .',
  0.380515376182179),
 ('inter rater reliability was excellent to good .', 0.6772256832296983),
 ('scholars and some journalists have an optimistic view.',
  0.4155700842603198),
 ('intra rater reliability was also excellent to good .', 0.8728829092339162)]

In [81]:
random_sentence_index=random.sample(range(0, len(sentences)), 1)
for i in random_sentence_index:
  x=similar_sentence(sentences,vectors,i)
x

The sentence is : we focused this study in two different iems urea cycle disorders and maple syrup urine disease .
The indices of the sentences most similar to sentence  1642  :  [2142 4392 2380 1065 2538  111 1369 2787 1642 4078]


[('all presented with high ck levels and liver disease .', 0.6666240835556198),
 ('we are already physically exhausted we can not tolerate any more if the condition persists.',
  0.6732830241345272),
 ('we report a new almost monosymptomatic neurological presentation.',
  0.6965934246330571),
 ('results we will present the first full length structure of human pah.',
  0.6735706253306915),
 ('we report a newborn with sucla mutations presenting with early neonatal neurological distress in the context of perinatal asphyxia.',
  0.692789981590422),
 ('what factors influence symptom reporting during an emerging infectious disease outbreak?',
  0.6860559919324207),
 ('we describe several cases of secondary bh deficiency related to onco haematological chemotherapy exploring the possible pathogenic mechanisms.',
  0.710951944748961),
 ('we have described that the c abl kinase is activated in other neurodegenerative diseases.',
  0.7111005251961884),
 ('we focused this study in two different ie

In [82]:
random_sentence_index=random.sample(range(0, len(sentences)), 1)
for i in random_sentence_index:
  x=similar_sentence(sentences,vectors,i)
x

The sentence is : in cases we have found common mutation p.vm in compound with another previously unknown pathogenic allele.
The indices of the sentences most similar to sentence  2325  :  [4895 3281 3035  208 3462 1059 3547 2353 2825 2325]


[('in late december the health authorities of wuhan city reported a series of pneumonia cases of unknown etiology.',
  0.7032458899119349),
 ('in addition he was also admitted for multiple episodes of fever of unknown origin.',
  0.703251233541794),
 ('in rabbits developmental toxicity was observed at maternally toxic doses.',
  0.7561739840759675),
 ('in addition patients are still under functional studies of which are new pathological entities.',
  0.7171675470317088),
 ('disparities in covid death are the result of structural racism.',
  0.7526005328787364),
 ('in humans hydroxylation of phe prevents its toxic accumulation.',
  0.7561502124253818),
 ('in fact even under this fixed set of assumptions there is substantial uncertainty in who exactly is infecting whom.',
  0.7451306889317485),
 ('in multiple disorders poor prognosis was predicted by earlier onset and truncating rather than missense mutations.',
  0.7056290205992962),
 ('in this study one novel mutation was described.', 

We can see in the results that while we are trying to get the most similar sentences with respect to a given sentence, the sentence itself arrives in the list and cosine similarity value is 1 with repect to itself. It proves that the cosine similarity measurement is working correctly. 

Now we would try to show the **most similar sentence pairs** across all the sentences in the corpus.

In [83]:
def most_similar(vectors,sentences,top = 5): # finding the most similar sentence pairs
  distance_array = []
  for i in tqdm(range(len(sentences))):
    for j in range(i+1, len(sentences)):
      dist = fastdist.cosine(vectors[i], vectors[j])
      if dist > 0.01 and dist<1:  #we are discarding similarity 1 because they are same sentences
        distance_array.append((dist, (i,j)))
  sorted_dist = sorted(distance_array, key = lambda x: -x[0])[:top]  #sorting the list based on distance
  sentence_pair = [(sentences[i[1][0]], sentences[i[1][1]], i[0]) for i in sorted_dist]
  return sentence_pair

In [84]:
_=most_similar(vectors,sentences)

100%|██████████| 5204/5204 [00:31<00:00, 166.62it/s] 


In [85]:
_

[('the mean age at diagnosis was years .',
  'results the mean age at diagnosis was .',
  0.9999999999999999),
 ('the analysis of mass cytometry of the peripheral blood mononuclear cells is described in supplementary .',
  'the analysis of the rna seq survey is described in supplementary .',
  0.9999999999999999),
 ('p .. data are mean values sem.',
  'p .. data are mean values sem.',
  0.9999999999999999),
 ('patients were male and were female.',
  'results patients were male and patients were female.',
  0.9999999999999997),
 ('bal fluid from acid treated mice induces large amounts of il in wt but not tlr alveolar macrophages.',
  'bal fluid from wt mice hr after a hn challenge triggers il in wt but not tlr alveolar macrophages.',
  0.9999999999999994)]